In [14]:
import pandas as pd

In [9]:
color_names = ["red", "green", "blue", "orange", "gray", "yellow"]
image_size = 75

In [54]:
def solve(question_latent, image_latent):
    
    color_i = -1
    for i in range(6):
        if question_latent[i]:
            color_i = i
            break
    color = color_names[color_i]
    
    if question_latent[6]:  # non-relational question
        
        if question_latent[8]:  # What shape is the {} object?
            for index, row in image_latent.iterrows():
                if row["Color"] == color:
                    return row["Shape"]
        elif question_latent[9]:  # Is the {} object on the left?
            for index, row in image_latent.iterrows():
                if row["Color"] == color:
                    return int(row["X"] < image_size / 2)
        elif question_latent[10]:  # Is the {} object on the top?
            for index, row in image_latent.iterrows():
                if row["Color"] == color:
                    return int(row["Y"] < image_size / 2)
        else:
            raise RuntimeError("Invalid Question Latent")
        
    elif question_latent[7]:  # relational question
        
        if question_latent[8]:  # What shape is the object closest to the {} object?
            main_object = None
            for index, row in image_latent.iterrows():
                if row["Color"] == color:
                    main_object = row
                    break
            image_latent["distance"] = image_latent.apply(lambda x: (x["X"] - main_object["X"])**2 + (x["Y"] - x["Y"])**2 if x["Color"] != main_object["Color"] else 9e10, axis=1)
            closest_object = image_latent.loc[image_latent["distance"].idxmin()]
            return closest_object["Shape"]
        elif question_latent[9]:  # What shape is the object furthest from the {} object?
            main_object = None
            for index, row in image_latent.iterrows():
                if row["Color"] == color:
                    main_object = row
                    break
            image_latent["distance"] = image_latent.apply(lambda x: (x["X"] - main_object["X"])**2 + (x["Y"] - x["Y"])**2 if x["Color"] != main_object["Color"] else -9e10, axis=1)
            farthest_object = image_latent.loc[image_latent["distance"].idxmax()]
            return farthest_object["Shape"]
        elif question_latent[10]:  # How many objects are the same shape as the {} object?
            main_object = None
            for index, row in image_latent.iterrows():
                if row["Color"] == color:
                    main_object = row
                    break
            same_shape_count = -1
            for index, row in image_latent.iterrows():
                if row["Shape"] == main_object["Shape"]:
                    same_shape_count += 1
            return same_shape_count
        else:
            raise RuntimeError("Invalid Question Latent")
        
    else:
        raise RuntimeError("Invalid Question Latent")

In [55]:
# set up an example image latent
ids = [1, 2, 3, 4, 5, 6]
Xs = [38, 21, 57, 35, 68, 8]
Ys = [52, 17, 49, 40, 63, 55]
colors = ["red", "green", "blue", "orange", "gray", "yellow"]
shapes = ["circle", "rectangle", "circle", "rectangle", "circle", "rectangle"]
image_latent = pd.DataFrame({"ID": ids, "X": Xs, "Y": Ys, "Color": colors, "Shape": shapes})

In [56]:
image_latent

,ID,X,Y,Color,Shape
0,1,38,52,red,circle
1,2,21,17,green,rectangle
2,3,57,49,blue,circle
3,4,35,40,orange,rectangle
4,5,68,63,gray,circle
5,6,8,55,yellow,rectangle


In [59]:
question_latent = [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]

In [60]:
solve(question_latent, image_latent)

'rectangle'